In [ ]:
import pandas as pd
import datetime

from lib.ts_backtester import Backtester
from strategies.rsi_2.S_rsi_plot import plot

from tradeexecutor.state.trade import TradeExecution
from tradeexecutor.strategy.pandas_trader.position_manager import PositionManager
from tradeexecutor.state.state import State
from tradingstrategy.universe import Universe

import pandas_ta as ta

In [ ]:
from tradingstrategy.timebucket import TimeBucket
from tradingstrategy.chain import ChainId

# Make sure that backtester is defined or not
backtester = Backtester(
    candle_time_bucket=TimeBucket.h4,
    stop_loss_time_bucket=TimeBucket.h1,
    trading_pair=[(ChainId.ethereum, "uniswap-v3", "WETH", "USDC", 0.0005)],
    start_at=datetime.datetime(2023, 1, 1),
    end_at=datetime.datetime(2023, 1, 4),
    reserve_currency="USDC",
)
# try:
#     backtester
# except NameError:
#     print("backtester is not defined")
#     backtester = Backtester(
#         timeframe=TimeBucket.h4,
#         trading_pair=("WBNB", "BUSD"),
#         chain_id=ChainId.bsc,
#         exchange_slug="pancakeswap-v2",
#     )
#     timeframe=TimeBucket.h4,
#     trading_pair=("WBNB", "BUSD"),
#     chain_id=ChainId.bsc,
#     exchange_slug="pancakeswap-v2",
# )

In [1]:
ma_long = 216
ma_short = 9
rsi_cutt = 13
atr_distance = 2


def get_signals(candles):
    close = candles["close"].iloc[-1]
    low = candles["low"].iloc[-1]

    # Calculate indicators
    sma_short = ta.sma(candles["close"], length=ma_short)
    sma_short = ta.sma(candles["close"], length=ma_short).iloc[-1]
    sma_long = ta.sma(candles["close"], length=ma_long).iloc[-1]
    rsi = ta.rsi(candles["close"], length=2).iloc[-1]
    atr = ta.atr(candles["high"], candles["low"], candles["close"], length=14).iloc[-1]

    # Calculate signals
    entry = close >= sma_long and rsi <= rsi_cutt
    exit = close > sma_short
    sl = low - atr * atr_distance
    sl_pct = sl * 100 / candles["open"].iloc[-1]

    indicators = {
        "sma_short": sma_short,
        "sma_long": sma_long,
        "rsi": rsi,
        "atr": atr,
    }

    return entry, exit, sl_pct, indicators


def calculate_size(state, close):
    cash = state.portfolio.get_current_cash()
    return cash * 0.99


def loop(timestamp, universe, state, pricing_model, cycle_debug_data):
    # The pair we are trading
    trades = []
    pair = universe.pairs.get_single()
    pair.fee = 0.0050

    candles: pd.DataFrame = universe.candles.get_single_pair_data(
        timestamp, sample_count=ma_long
    )
    current_price = candles["close"].iloc[-1]

    entry, exit, sl, indicators = get_signals(candles)

    # Create a position manager helper class that allows us easily to create
    # opening/closing trades for different positions
    position_manager = PositionManager(timestamp, universe, state, pricing_model)
    buy_amount = calculate_size(state, current_price)

    if not position_manager.is_any_open():
        if entry:
            trades += position_manager.open_1x_long(pair, buy_amount, stop_loss_pct=sl)
    else:
        if exit:
            trades += position_manager.close_all()

    # plot(state, timestamp, indicators)

    return trades


start_at = datetime.datetime(2023, 1, 1)
end_at = datetime.datetime(2023, 1, 4)


backtester.backtest(start_at, end_at, loop)
# backtester.stats()
# backtester.general_stats()
backtester.plot()

2023-01-04 00:00:00
---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
Cell In[11], line 73
     69 start_at = datetime.datetime(2023, 1, 1)
     70 end_at = datetime.datetime(2023, 1, 4)
---> 73 backtester.backtest(start_at, end_at, loop)
     74 # backtester.stats()
     75 # backtester.general_stats()
     76 backtester.plot()

File ~/workspace/trading_strategy/ts-startegy/lib/ts_backtester.py:123, in Backtester.backtest(self, start_at, end_at, decide_trades, initial_deposit, cycle_duration, reserve_currency, trade_routing)
    120 self.end_at = end_at
    121 print(end_at)
--> 123 state, _, debug_dump = run_backtest_inline(
    124     name="BNB/USD EMA crossover example",
    125     start_at=start_at,
    126     end_at=end_at,
    127     client=client,
    128     cycle_duration=cycle_duration,
    129     decide_trades=decide_trades,
    130     universe=self.universe,
    131 

In [ ]:
from tradeexecutor.analysis.trade_analyser import build_trade_analysis
from IPython.core.display_functions import display

analysis = build_trade_analysis(backtester.state.portfolio)
from tradeexecutor.analysis.trade_analyser import expand_timeline

timeline = analysis.create_timeline()

expanded_timeline, apply_styles = expand_timeline(
    backtester.universe.universe.exchanges, backtester.universe.universe.pairs, timeline
)

expanded_timeline.drop(
    columns=[
        "Id",
        "Remarks",
        "Exchange",
        "Trade count",
        "Duration",
        "Base asset",
        "Quote asset",
        "PnL %",
        "PnL % raw",
    ],
    inplace=True,
)
expanded_timeline.head()